# Import Libraries

In [ ]:
import csv
import os
from sqlalchemy.orm import Session
from backend.app.models import Ingredient, Recipe, RecipeIngredient, User, RecipeTag, FridgeItem, FridgeLog
from backend.app.database import SessionLocal, Base
from backend.app.database import engine

# Helper Functions

In [ ]:
def parse_ingredients(ingredient_str):
    if not ingredient_str or not ingredient_str.strip():
        return None, None, None
    parts = ingredient_str.split(',')
    if len(parts) < 2:
        return None, None, None
    name_unit = parts[0].strip()
    quantity = parts[1].strip()
    if '[' in name_unit and ']' in name_unit:
        name, unit = name_unit.split('[')
        name = name.strip()
        unit = unit.replace(']', '').strip()
    else:
        name = name_unit.strip()
        unit = ''
    try:
        quantity = float(quantity.replace('/', '.'))
    except Exception:
        quantity = None
    return name, unit, quantity

# Set Up Database
Set up the database connection and create tables using SQLAlchemy's `Base.metadata.create_all`.

In [3]:
Base.metadata.create_all(bind=engine)
db: Session = SessionLocal()

In [4]:
# Clear all tables in the specified order
db.query(RecipeIngredient).delete()
db.query(RecipeTag).delete()
db.query(FridgeItem).delete()
db.query(FridgeLog).delete()
db.query(Ingredient).delete()
db.query(Recipe).delete()
db.query(User).delete()
db.commit()
print("All tables cleared.")

All tables cleared.


# Parse CSV and Populate Database
Read the English CSV file, parse its contents, and populate the database with recipes, ingredients, and their relationships.
print("All tables cleared.")

In [5]:
CSV_PATH = '../Planner - Recipes (EN).csv'

with open(CSV_PATH, newline='', encoding='utf-8') as csvfile:
    reader = list(csv.reader(csvfile))
    # First row: headers (Name, Alfredo Pasta, ...)
    recipe_names = reader[0][1:]
    # Find the row indices for Ingredients and Recipe (instruction link)
    ingredients_rows = [i for i, row in enumerate(reader) if row[0].strip() == 'Ingredients']
    recipe_link_rows = [i for i, row in enumerate(reader) if row[0].strip() == 'Recipe']
    yt_link_rows = [i for i, row in enumerate(reader) if row[0].strip() == 'YT']
    if not ingredients_rows:
        print('No Ingredients row found!')
    else:
        ingredients_start = ingredients_rows[0]
        # Build ingredient and recipe tables
        ingredient_map = {}  # name -> Ingredient
        recipe_map = {}      # name -> Recipe

        # Prepare instruction links
        instruction_links = {}
        if recipe_link_rows:
            recipe_links = reader[recipe_link_rows[0]][1:]
        else:
            recipe_links = [None] * len(recipe_names)
        if yt_link_rows:
            yt_links = reader[yt_link_rows[0]][1:]
        else:
            yt_links = [None] * len(recipe_names)
        for col, recipe_name in enumerate(recipe_names):
            if not recipe_name.strip():
                continue
            # Prefer Recipe link, fallback to YT link
            instruction_link = recipe_links[col] if recipe_links[col] and recipe_links[col].strip() else yt_links[col] if yt_links[col] and yt_links[col].strip() else None
            # Insert recipe
            recipe = db.query(Recipe).filter_by(name=recipe_name).first()
            if not recipe:
                recipe = Recipe(name=recipe_name, instruction_link=instruction_link)
                db.add(recipe)
                db.commit()
                db.refresh(recipe)
            else:
                # Update instruction_link if needed
                if instruction_link and getattr(recipe, 'instruction_link', None) != instruction_link:
                    recipe.instruction_link = instruction_link
                    db.commit()
            recipe_map[recipe_name] = recipe
        # Now, for each ingredient row, add ingredients and recipe_ingredients
        for row in reader[ingredients_start+1:]:
            if not any(row):
                continue
            for col, cell in enumerate(row[1:]):
                recipe_name = recipe_names[col]
                if not recipe_name.strip() or not cell.strip():
                    continue
                name, unit, quantity = parse_ingredients(cell)
                if not name:
                    continue
                # Insert ingredient if not exists
                ingredient = db.query(Ingredient).filter_by(name=name).first()
                if not ingredient:
                    ingredient = Ingredient(name=name, default_unit=unit)
                    db.add(ingredient)
                    db.commit()
                    db.refresh(ingredient)
                ingredient_map[name] = ingredient
                # Insert recipe_ingredient
                recipe = recipe_map[recipe_name]
                exists = db.query(RecipeIngredient).filter_by(recipe_id=recipe.id, ingredient_id=ingredient.id).first()
                if not exists:
                    ri = RecipeIngredient(recipe_id=recipe.id, ingredient_id=ingredient.id, quantity=quantity or 0, unit=unit or ingredient.default_unit or "")
                    db.add(ri)
        db.commit()

# Run the Script
Execute the main logic to ensure the database is populated correctly and print a completion message.

In [ ]:
print("Import complete.")